In [1]:
import pandas as pd
import numpy as np

import pickle 

import re
import string

import nltk
#nltk.download()

from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.metrics.pairwise import cosine_similarity

import spacy
from spacy import displacy
from spacy.symbols import amod
from spacy.lang.en.stop_words import STOP_WORDS

from collections import Counter
from spacy.symbols import amod
from pprint import pprint

import scattertext as st

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import pyLDAvis
import pyLDAvis.sklearn

# Get Data

In [11]:
with open('ted_video_comments_2193.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [16]:
df.head()

,video_id,comments
0,-2Dj9M71JAc,She sounds like the person before was singing ...
1,-3y6_7_5PcQ,Stop funding large pharmaceuticals by buying t...
2,-7ORAKULel4,"False, the technology is still out of reach fo..."
3,-BvcToPZCLI,"What is the carbon footprint of your food?, sh..."
4,-E5K2pGB08Y,"Full of Karens., stumbled here through autopla..."


In [13]:
len(df)

2193

In [14]:
with open('ted_video_stats_2722.pkl', 'rb') as picklefile:
    stats = pickle.load(picklefile)

In [10]:
len(stats)

2722

In [55]:
nrc = pd.read_csv('nrc.csv')

In [57]:
nrc.head()

,Unnamed: 0,word,sentiment
0,1,abacus,trust
1,2,abandon,fear
2,3,abandon,negative
3,4,abandon,sadness
4,5,abandoned,anger


In [56]:
afinn = pd.read_csv('afinn.csv')

In [59]:
afinn.head()

,Unnamed: 0,word,value
0,1,abandon,-2
1,2,abandoned,-2
2,3,abandons,-2
3,4,abducted,-2
4,5,abduction,-2


# Text Preprocessing

### Formatting

In [15]:
df.comments = df.comments.str.replace('{', '').replace('}', '')

In [29]:
df1 = pd.merge(df, stats,  how='left', left_on='video_id', right_on = 'video_id')

### SpaCy

In [31]:
nlp = spacy.load('en_core_web_sm')

In [17]:
#doc = nlp(df['comments'].values[0])

In [80]:
#test = doc[:14]

In [17]:
#for token in test:
#    print(token.text, token.pos_, token.lemma_, token.is_stop)

In [51]:
#def is_noun_adv_adj_verb(token):
#    if token.doc.is_tagged is False:
#        raise ValueError('token not POS-tagged')
#    return (token.pos_ == 'NOUN') | (token.pos_ == 'ADJ') | (token.pos_ == 'ADV') | (token.pos_ == 'VERB' )#

In [28]:
# mask = [is_noun_adv_adj_verb(word) for word in test]
# len(mask)

In [ ]:
# a = np.array(a) # convert list to numpy array
# >> result = a[msk] # mask a
# >> result.tolist()

In [19]:
#from spacy.tokens import Token

#def get_is_excluded(token):
#    # Getter function to determine the value of token._.is_excluded
#    return token.pos_ in ['some', 'excluded', 'words']

#Token.set_extension('is_excluded', getter=get_is_excluded)

In [22]:
#test.text

In [20]:
#for token in test:
 #   print(token.text, token.dep_)

In [21]:
#displacy.render(test, style='dep', options={'distance': 120})

In [23]:
#for token in test:
#    if token.dep_ == 'advmod':
#        print(f"posessive: {token.text} --> MODIFY: {token.head}")

In [24]:
#spacy.explain("advmod")

In [25]:
#for ent in doc.ents:
#    print(ent.text, ent.label_)

In [26]:
#spacy.explain("PERSON")

In [27]:
#displacy.render(doc, style='ent', jupyter=True)

In [84]:
#stemmer=LancasterStemmer()
#porter=PorterStemmer()

In [132]:
# englishStemmer=SnowballStemmer("english", ignore_stopwords=True)
# def stemComment(comment_line):
#     token_words=word_tokenize(comment_line)
#     stem_line=[]
#     for word in token_words:
#         stem_line.append(englishStemmer.stem(word))
#         stem_line.append(" ")
#     return "".join(stem_line)

In [133]:
# stop_words = set(stopwords.words('english'))

# def remove_stop_words(comment_line):
#     token_words=word_tokenize(str(comment_line))
#     filtered_comment_line = [w for w in token_words if not w in stop_words] 
    
#     return " ".join(filtered_comment_line)

In [61]:
# alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
# punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

In [137]:
# df['comments'] = df['comments'].map(remove_stop_words).map(alphanumeric).map(punc_lower)

In [138]:
# #https://stackoverflow.com/questions/29270917/removing-custom-stop-words-form-a-phrase-in-python
# my_stop_words_lst = ['ted', 'text', 'thank', 'you']

# for w in my_stop_words_lst:
#     pattern = r'\b'+w+r'\b'
#     custom_stop = lambda x: re.sub(pattern, ' ', x)
#     df['comments'] = df['comments'].map(custom_stop)

In [139]:
# df['comments'] = df['comments'].map(stemComment)

In [60]:
#df.head()

# EDA

### Scattertext

In [103]:
 corpus = st.CorpusFromPandas(df1,
                              category_col='success',
                              text_col='comments',
                              nlp=st.whitespace_nlp_with_sentences
                             ).build()

In [104]:
 html = st.produce_scattertext_explorer(
         corpus,
         category='yes',
         category_name='Likable',
         not_category_name='Dislikable',
         minimum_term_frequency=10,
         pmi_threshold_coefficient=5,
         width_in_pixels=1000,
         #metadata=df1['prime_tag']
         metadata=corpus.get_df()['prime_tag']
         )

In [105]:
open('comments.html', 'wb').write(html.encode('utf-8'));

### LDA PyViz

In [127]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=1000,                # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(df.comments)

In [128]:
n_components = 10

lda_model = LatentDirichletAllocation(n_components=n_components,   # Number of topics
                                      max_iter=10,                 # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,            # Random state
                                      #batch_size=128,             # n docs in each learning iter
                                      n_jobs = -1,                 # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [129]:
pyLDAvis.enable_notebook()

panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
panel

/Users/elena/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.053859  0.015955       1        1  24.777800
8     -0.071851 -0.008928       2        1  19.559825
7     -0.071295  0.024468       3        1  14.202735
4     -0.079318  0.041838       4        1  11.950374
5     -0.063053  0.058107       5        1  11.334740
6     -0.070162  0.058874       6        1   6.997485
1     -0.070246  0.098449       7        1   5.394002
3     -0.064671 -0.019688       8        1   4.081690
0      0.472794  0.094732       9        1   0.890047
2      0.071662 -0.363807      10        1   0.811303, topic_info=    Category         Freq     Term        Total  loglift  logprob
152  Default  3761.000000      com  3761.000000  30.0000  30.0000
992  Default  3272.000000      www  3272.000000  29.0000  29.0000
429  Default  3488.000000    https  3488.000000  28.0000  28.0000
426  Default  3429.000000     href  3429.000000  27.0000  27.0000
999  Default  2830.000000  youtube  2830.000000  26.0000  26.0000
..       ...          ...      ...          ...      ...      ...
840  Topic10    26.836698     stop   873.551783   1.3315  -4.6694
552  Topic10    30.062238     make  2100.704394   0.5675  -4.5559
519  Topic10    30.020979     like  5109.653225  -0.3227  -4.5572
517  Topic10    27.654593     life  2087.315433   0.4904  -4.6393
476  Topic10    26.933911     just  4455.323969  -0.2942  -4.6657

[552 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2         1  0.103075     2019
2         2  0.206149     2019
2         4  0.198787     2019
2         5  0.287137     2019
2         6  0.080987     2019
...     ...       ...      ...
999       4  0.031802  youtube
999       5  0.874548  youtube
999       6  0.019081  youtube
999       7  0.003534  youtube
999       8  0.002827  youtube

[2317 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 8, 5, 6, 7, 2, 4, 1, 3])

# Sentiment Analysis 

### Vader

In [52]:
analyser = SentimentIntensityAnalyzer()
def extract_polarity(text, polarity_type = 'neg'):
    all_val = analyser.polarity_scores(text)
    return all_val[polarity_type]

In [53]:
df['polarity_neg_comments'] = df['comments'].map(extract_polarity)
df['polarity_pos_comments'] = df['comments'].apply(extract_polarity, polarity_type = 'pos')
df['polarity_neu_comments'] = df['comments'].apply(extract_polarity, polarity_type = 'neu')
df['polarity_compound_comments'] = df['comments'].apply(extract_polarity, polarity_type = 'compound')

### SpaCy: Vocab on Polarity

In [32]:
df1['spacy_doc'] = list(nlp.pipe(df1.comments))

In [49]:
#df1.head()

In [38]:
success = df1[df1.success=='yes']
no_success = df1[df1.success=='no']

In [39]:
pos_adj = [token.text.lower() for doc in success.spacy_doc for token in doc if token.pos_=='ADJ']
neg_adj = [token.text.lower() for doc in no_success.spacy_doc for token in doc if token.pos_=='ADJ']

pos_noun = [token.text.lower() for doc in success.spacy_doc for token in doc if token.pos_=='NOUN']
neg_noun = [token.text.lower() for doc in no_success.spacy_doc for token in doc if token.pos_=='NOUN']

In [43]:
Counter(pos_noun).most_common(12)

[('people', 2450),
 ('time', 1035),
 ('talk', 994),
 ('life', 975),
 ('video', 936),
 ('world', 832),
 ('way', 814),
 ('years', 762),
 ('things', 617),
 ('work', 513),
 ('man', 488),
 ('don&#39;t', 470)]

In [44]:
Counter(neg_noun).most_common(12)

[('people', 3745),
 ('world', 1240),
 ('talk', 1079),
 ('time', 1046),
 ('way', 1044),
 ('life', 997),
 ('video', 868),
 ('women', 817),
 ('years', 792),
 ('don&#39;t', 727),
 ('money', 691),
 ('things', 620)]

In [47]:
def get_amods(noun, ser):
    amod_list = []
    for doc in ser:
        for token in doc:
            if (token.text) == noun:
                for child in token.children:
                    if child.dep == amod:
                        amod_list.append(child.text.lower())
    return sorted(amod_list)

def amods_by_sentiment(noun):
    print(f"Adjectives describing {str.upper(noun)}:\n")
    
    print("POSITIVE:")
    pprint(get_amods(noun, success.spacy_doc))
    
    print("\nNEGATIVE:")
    pprint(get_amods(noun, no_success.spacy_doc))

In [50]:
#amods_by_sentiment("talk") ###may be i should tune the threshold

### Sentiment_vocab

In [62]:
df.head()

,video_id,comments,polarity_neg_comments,polarity_pos_comments,polarity_neu_comments,polarity_compound_comments
0,-2Dj9M71JAc,She sounds like the person before was singing ...,0.167,0.127,0.707,-0.9916
1,-3y6_7_5PcQ,Stop funding large pharmaceuticals by buying t...,0.118,0.156,0.726,0.9915
2,-7ORAKULel4,"False, the technology is still out of reach fo...",0.109,0.136,0.755,0.8998
3,-BvcToPZCLI,"What is the carbon footprint of your food?, sh...",0.050,0.089,0.862,0.9986
4,-E5K2pGB08Y,"Full of Karens., stumbled here through autopla...",0.018,0.212,0.769,0.9835


In [108]:
nrc_trust

,Unnamed: 0,word,sentiment
0,1,abacus,trust
14,15,abbot,trust
69,70,absolution,trust
78,79,abundance,trust
92,93,academic,trust
...,...,...,...
13806,13807,wot,trust
13866,13867,yearning,trust
13892,13893,zeal,trust
13895,13896,zealous,trust


In [106]:
#df.comments[1]

In [109]:
words = df.comments.values[0].split()

In [116]:
def get_emotion_score(text, emotion = 'trust'):
    
    nrc_trust = nrc[nrc['sentiment'] == emotion]
    
    words = text.split()
    
    emotion_count = 0
    for word in words:
        if word in list(nrc_trust['word']):
            #print(word)
            emotion_count +=1
            
    return emotion_count

In [117]:
get_emotion_score(df.comments.values[0], emotion = 'trust')

18

In [119]:
nrc_emotions = nrc['sentiment'].unique()

In [135]:
nrc_emotions = nrc['sentiment'].unique()

for nrc_emotion in nrc_emotions:
    df[nrc_emotion] = df['comments'].apply(get_emotion_score, emotion = nrc_emotion)

In [138]:
df.head()

,video_id,comments,polarity_neg_comments,polarity_pos_comments,polarity_neu_comments,polarity_compound_comments,trust,fear,negative,sadness,anger,surprise,positive,disgust,joy,anticipation
0,-2Dj9M71JAc,She sounds like the person before was singing ...,0.167,0.127,0.707,-0.9916,0.008208,0.005928,0.012312,0.008208,0.007752,0.003648,0.012768,0.005016,0.005472,0.007296
1,-3y6_7_5PcQ,Stop funding large pharmaceuticals by buying t...,0.118,0.156,0.726,0.9915,0.009576,0.005472,0.010488,0.005928,0.005016,0.001824,0.018696,0.003192,0.005928,0.005928
2,-7ORAKULel4,"False, the technology is still out of reach fo...",0.109,0.136,0.755,0.8998,0.004560,0.003192,0.005472,0.001368,0.001824,0.000456,0.006384,0.000912,0.001368,0.001368
3,-BvcToPZCLI,"What is the carbon footprint of your food?, sh...",0.050,0.089,0.862,0.9986,0.012768,0.012768,0.023256,0.008664,0.006384,0.002736,0.038304,0.004560,0.004560,0.010032
4,-E5K2pGB08Y,"Full of Karens., stumbled here through autopla...",0.018,0.212,0.769,0.9835,0.000456,0.000000,0.000456,0.000000,0.000000,0.000912,0.001368,0.000000,0.000912,0.000912


In [137]:
for nrc_emotion in nrc_emotions:
    df[nrc_emotion] = df[nrc_emotion]/len(df['comments'])

In [141]:
with open('ted_video_comments_emotions.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [140]:
!pwd

/Users/elena/Desktop/Metis/Project_4_Ted/Project-4-Ted
